<a href="https://colab.research.google.com/github/edrian-liao/harvestnet/blob/main/notebooks/HarvestNet_Evaluate_Model_Subdivisions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install wandb -qU
!pip install torchmetrics -qU
!pip install accelerate -qU

In [4]:
import os
import time
import torch
from torch.utils.data import DataLoader
import torchvision
import wandb
import numpy as np
from torchmetrics import Accuracy, F1Score, AUROC, Precision, Recall
from accelerate import Accelerator
from collections import defaultdict
import cv2

In [5]:
from tools.dataset import SkysatLabelled
from tools.config import Config_Resnet

In [ ]:
class ResNet50(torch.nn.Module):
    def __init__(self, num_classes, model_weights_path):
        super(ResNet50, self).__init__()
        self.resnet50 = torchvision.models.resnet50()
        num_features = self.resnet50.fc.out_features
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(num_features, num_classes),
            torch.nn.Sigmoid(),
        )
        # Load the state dictionary
        state_dict = torch.load(model_weights_path)
        # Assign the loaded state dictionary to the model
        self.load_state_dict(state_dict)

    def forward(self, x):
        logits = self.resnet50(x)
        return self.fc(logits)

# Example usage:
num_classes = 10  # Number of output classes
model_weights_path = 'best_resnet.pth'
model = ResNet50(num_classes, model_weights_path)